In [1]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pandas as pd
import numpy as np
from random import randint,shuffle

In [2]:
d=open("dictionary.txt", 'r', encoding='utf-8')
s=open("sentiment_labels.txt", encoding='utf-8')
sents=[]
labels=[]
for line in d:
    sents.append(line.split('|')[0])
for line in s:
    score=line.split('|')[1]
    labels.append(score.split('\n')[0])
d.close()
s.close()
print('sents', len(sents))

sents 239232


In [3]:
def pad_vector(size, sent_feat_temps):
    return [0]*(size-len(sent_feat_temps)) + sent_feat_temps
def create_vocab(fts_list):#takes a list of lists of words
    rV=[]
    for el in fts_list:
        for word in el:
            rV.extend(el)
    rV=set(rV)
    return ['<pad>'] + list(rV)

In [4]:
token_sents=[]
lemmatizer = WordNetLemmatizer()
for sent in sents:
    sentence=word_tokenize(sent)
    lem_sentence=[]
    for word in sentence:
        lem_sentence.append(lemmatizer.lemmatize(word.lower()))
    token_sents.append(lem_sentence)

#    print(sentence)
#    input()
zipped=list(zip(token_sents,labels))
del zipped[0]#getting rid of the header
zipped.sort(key = lambda x: len(x[0]), reverse=True)
language_data=zipped[:150000]
#list of tuples. One per sentence. ([list of lemmatized lowercase words in the sentence], a sentiment score)

sentlist=[sent for sent,score in language_data]
longest_sent = max(len(sent) for sent in sentlist)
vocab=create_vocab(sentlist)
vocabsize=len(vocab)
inttofeat = dict(zip(range(vocabsize), vocab))
feattoint = dict(zip(vocab, range(vocabsize)))

print('language_data', len(language_data))

padded_data=[]
for sent,score in language_data:
    feat=[feattoint[word] for word in sent]
    padded_data.append((pad_vector(longest_sent, feat),score))

print('padded_data', len(padded_data))

language_data 150000
padded_data 150000


In [5]:
#Now, sort the sentences in 3 categories:

angry_sents=[]
happy_sents=[]
#neutral_sents=[]

for sent,score in padded_data:
    if 0.0 <= float(score) < 0.5:
        angry_sents.append((sent,'NEG'))
#    elif float(score) == 0.50:
#        neutral_sents.append((sent,score))
    elif 0.55 <= float(score) <= 1:
        happy_sents.append((sent,'POS'))
        
print('negative sentences: ', len(angry_sents))
print('positive sentences: ', len(happy_sents))
#print('neutral', len(neutral_sents))

negative sentences:  54364
positive sentences:  55674


In [6]:
np.save("angry_sents.npy", angry_sents)
np.save("happy_sents.npy", happy_sents)

In [8]:
#df=pd.read_csv("fer2013/fer2013.csv")#image data

##0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral
#angry_df=df[df.emotion == 0]
#happy_df=df[df.emotion == 3]
##neutral_df=df[df.emotion == 6]


In [9]:
#save to folder as png-files
#count=0
#for row in angry_df.iterrows():
#    image_feat=angry_df.iloc[count]['pixels'].split()
#    image_feat = [int(i) for i in image_feat]
#    new=np.array(image_feat).reshape((48,48)).astype(np.uint8)
#    im = Image.fromarray(new)
#    im.save('pngs/angry/output{}.png'.format(count))
#    count=count+1
    
#count=0
#for row in happy_df.iterrows():
#    image_feat=happy_df.iloc[count]['pixels'].split()
#    image_feat = [int(i) for i in image_feat]
#    new=np.array(image_feat).reshape((48,48)).astype(np.uint8)
#    im = Image.fromarray(new)
#    im.save('pngs/happy/output{}.png'.format(count))
#    count=count+1

In [11]:
import os
os.environ["CUDA_DEVICE_ORDER"]= "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [12]:
import numpy as np
from keras.applications.resnet50 import ResNet50
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image as kimage

Using TensorFlow backend.


In [13]:
# If you are running this for the first time in this machine, keras will download the pre-trained weights.
pretrained_cnn_model = ResNet50(weights='imagenet', include_top=False)

In [ ]:
pretrained_cnn_model.summary()

In [14]:
def img2vec(image_path):
    x = kimage.load_img(image_path, target_size=[224,224])
    x_array = kimage.img_to_array(x)
    xs_array = np.array([x_array,])
    # notice that we are not using full capacity of the GPU when we are passing only one image per prediction.
    # we could have a larger batch.
    return pretrained_cnn_model.predict(preprocess_input(xs_array)).flatten()

In [15]:
import glob

In [31]:
angry_images=[]
happy_images=[]

path = "pngs/happy/output*.png"
for png in glob.glob(path):
    happy_images.append(img2vec(png))
    
path = "pngs/angry/output*.png"
for png in glob.glob(path):
    angry_images.append(img2vec(png))
    

In [32]:
print('angry images: ', len(angry_images))
print('happy images: ', len(happy_images))

angry images:  4953
happy images:  8989


In [52]:
np.save("angry_images.npy", angry_images)
np.save("happy_images.npy", happy_images)

In [42]:
all_data=[]

sent_count=0
for image in angry_images:
    
    for step in range(6):
        all_data.append((image, angry_sents[sent_count][0], 'NEG'))
        sent_count=+1
        
sent_count=0        
for image in happy_images:
    for step in range(5):
        all_data.append((image, happy_sents[sent_count][0], 'POS'))
        sent_count=+1            
    
#for sent,score in angry_sents:
#    for step in range(len(angry_sents)-1):
#        image_feat=angry_df.iloc[step]['pixels'].split()
#        image_feat = [int(i) for i in image_feat]
#        all_data.append((np.array(image_feat),sent,'NEG'))
                    
#for sent,score in happy_sents:#[:7500]:
#    image_feat=happy_df.iloc[randint(0, len(happy_df)-1)]['pixels'].split()
#    image_feat = [int(i) for i in image_feat]
#    all_data.append((np.array(image_feat),sent,'POS'))

#for sent,score in neutral_sents:
#    image_feat=neutral_df.iloc[randint(0, len(neutral_df)-1)]['pixels'].split()
#    image_feat = [int(i) for i in image_feat]
#    all_data.append((np.array(image_feat),sent,'NEU'))
print('total pairs: ',len(all_data))

total pairs:  74663


In [43]:
shuffle(all_data)

In [44]:
s=int((len(all_data)*0.8))#splits with 80% training, 20% test
train, test=all_data[:s], all_data[s:]

In [45]:
xtrain=[(image,sent) for (image,sent,label) in train]
ytrain=[label for (image,sent,label) in train]
xtest=[(image,sent) for (image,sent,label) in test]
ytest=[label for (image,sent,label) in test]

In [46]:
image_size=len(all_data[0][0])
sent_size=len(all_data[0][1])
labels=set(list(l for (i,s,l) in all_data))

xtrain2 = list(zip(*xtrain))
xtrain2 = [np.array(xtrain2[1]), np.array(xtrain2[0])]

xtest2 = list(zip(*xtest))
xtest2 = [np.array(xtest2[1]), np.array(xtest2[0])]

In [47]:
cat_codes = {
    'POS': 0,
    'NEU': 1,
    'NEG': 2,
}

In [48]:
ytrain2 = np.array([cat_codes[c] for c in ytrain])
ytest2 = np.array([cat_codes[c] for c in ytest])

In [49]:
np.save('parameters.npy',[image_size, sent_size]) 
np.save("vocab.npy", vocab)
np.save("xtrain0.npy", xtrain2[0])
np.save("xtrain1.npy", xtrain2[1])
np.save("ytrain.npy", ytrain2)
np.save("xtest0.npy", xtest2[0])
np.save("xtest1.npy", xtest2[1])
np.save("ytest.npy", ytest2)

In [50]:
sent_size

56